In [36]:
import random
import warnings
warnings.filterwarnings("ignore")
import math
import os
import time
import random
import shutil
import numpy as np
from scipy import spatial as st 
import pandas as pd
import timeit
import logging
import csv 

#####################################################################################
#####################################################################################
#############-----------variable initialization----------###################
                       # Initialize basic variables #
# this is the name of the folder with the data
mainFolder = "datafile"

# range of values for beta (heterogeneity of food quantity)
betas = [2.5]

# range of values for the forager population (tested against every beta)
foragers = [100] # [ 1, 5, 10, 50, 100, 200 ] kj comment out to test code

# number of files created for each program with different initial conditions 
# helps calculate averages
runs = 1

# number of time-steps for each run
steps = 10

# number of patches per run 
patches = 50000 

# number of patches in the memory of each forager at t = 0
# recall that memory > time-steps and memory <= patches, otherwise we get an error
memory = 50000 # tried to change memory down and it didn't work

## foraging radius
## whole landscape ranges from 0 to 1
fRadius = [0.1, 0.01, 0.001] # 1 seems like it wouldn't work because of check_forhome_avoidoverlap
global all_movdis
all_movdis =[]

##effective radius for moving to a new home location
er = 2


#####################################################################################
# set up logging
logging.basicConfig(filename='model.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


num_species = 1700
species = [f"specie_{i}" for i in range(num_species)]

diversity_beta = [1.5, 2.5, 3.5, 4.5]

try_per_day = 2

species_per_patch = 1 # in percentage, 0 to 1


agent_needs = {
    "EAT": {
        "eatpro": [],
        "eatfat": [],
        "eatcarb": [],
    },
    "ILLNESS": {
        "treatpain": [],
        "treatwound": [],
        "treatfever": [],
    },
     "SHELTER": {
        "needclothes": [],
        "needshoes": [],
        "needwood": [],
    },
     "TOOLS": {
        "needbasket": [],
        "needaxe": [],
        "needarrow": []
    }
}

#####################################################################################

key_to_category = {
    'eatpro': 'EAT',
    'eatfat': 'EAT',
    'eatcarb': 'EAT',
    'treatpain': 'ILLNESS',
    'treatwound': 'ILLNESS',
    'treatfever': 'ILLNESS',
    'needclothes': 'SHELTER',
    'needshoes': 'SHELTER',
    'needwood': 'SHELTER',
    'needbasket': 'TOOLS',
    'needaxe': 'TOOLS',
    'needarrow': 'TOOLS'
}

category_perc = {
    "EAT": 1,
    "ILLNESS": 1,
    "SHELTER": 1,
    "TOOLS": 1,
}
# changed to almost 1 to test the process, but then put back
# these numbers are close to real, as close as i can get

needs_perc = {}

# Set each need in a category to a percentage of the category 
# So each need is an equal percentage of the category
for category in agent_needs:
    num_needs = len(agent_needs[category])
    for need in agent_needs[category]:
        # divide category perc and divide by number of needs so that each need is an equal percentage of the category
        needs_perc[need] = category_perc[category] #/ num_needs 


efficacy = pd.DataFrame(columns=species, index=[key for cat in agent_needs for key in agent_needs[cat].keys()])
efficacy.index.name = "need"

for need in efficacy.index:
    #  generate number of species that are effective for this need
    num_effective = needs_perc[need] * num_species
    # get random columns from num_effective which are the chosen 'effective' species
    index_effective = random.sample(species, int(num_effective))
    
    # fill efficacy for each species for each need with random values from 0 to 1
    for index in index_effective:
        efficacy.loc[need, index] = random.uniform(0, 1)

# Fill in the rest of the remaining dataframe with 0
efficacy.fillna(0, inplace=True)
display(efficacy)


# test that it was done correctly
# and create a dict of species effective for each need

need_species_dict = {}
for need in efficacy.index:
    # filter to just the row for this need
    need_df = efficacy.loc[need]
    need_species = []
    need_species_efficacy = []
    # loop through all the columns (species), except 'need'
    for specie in [specie for specie in efficacy.columns if specie != 'need']:
        # find the efficacy number for that species
        specie_effect = pd.to_numeric(need_df[specie])
        if specie_effect > 0:
            # append non-zero efficacy species to need_species + efficacy amount to need_efficacy
            need_species.append(specie)
            need_species_efficacy.append(specie_effect)
    need_species_dict[need] = {
        'species': need_species,
        'efficacy': need_species_efficacy
    }

    # print, to test that it was done correctly
    print(f"Need: {need}, Num species: {len(need_species)}, Species effective for this need: {need_species}")



### helper funcs
def any_list_empty(agent_needs):
    for category in agent_needs:
        for need in agent_needs[category]:
            if len(agent_needs[category][need]) == 0:
                return True
    return False

def all_list_empty(agent_needs):
    for category in agent_needs:
        for need in agent_needs[category]:
            if len(agent_needs[category][need]) > 0:
                return False
    return True

def list_needs(agent_needs):
    needs_listed = []
    for category, needs in agent_needs.items():
        for need, species in needs.items():
            needs_listed.append(need)
    return needs_listed

needs_listed = list_needs(agent_needs)


,specie_0,specie_1,specie_2,specie_3,specie_4,specie_5,specie_6,specie_7,specie_8,specie_9,...,specie_1690,specie_1691,specie_1692,specie_1693,specie_1694,specie_1695,specie_1696,specie_1697,specie_1698,specie_1699
need,,,,,,,,,,,,,,,,,,,,,
eatpro,0.735937,0.131972,0.350469,0.8492,0.469909,0.302381,0.740236,0.899046,0.037107,0.144385,...,0.765454,0.06313,0.418764,0.394609,0.914722,0.225461,0.798187,0.400892,0.86152,0.438329
eatfat,0.582895,0.887276,0.710603,0.196123,0.228258,0.442826,0.13788,0.730772,0.786147,0.152186,...,0.70106,0.541128,0.464379,0.885775,0.055541,0.737023,0.785661,0.031413,0.521255,0.966802
eatcarb,0.700653,0.299866,0.114661,0.821445,0.836528,0.382995,0.118107,0.49277,0.306386,0.670569,...,0.896988,0.762318,0.460029,0.996754,0.985949,0.223074,0.807589,0.28736,0.261721,0.710942
treatpain,0.531306,0.914353,0.290974,0.088488,0.193779,0.862176,0.844556,0.391806,0.461923,0.555356,...,0.108613,0.529595,0.864924,0.599381,0.287259,0.291018,0.985445,0.242275,0.749355,0.82203
treatwound,0.835597,0.551788,0.68175,0.526886,0.624762,0.742341,0.954123,0.843254,0.06374,0.701366,...,0.43652,0.755146,0.587693,0.16224,0.332048,0.45993,0.568286,0.079552,0.94762,0.828373
treatfever,0.875901,0.931946,0.581064,0.500155,0.45317,0.487666,0.60955,0.103905,0.052741,0.564648,...,0.593469,0.544794,0.781015,0.498686,0.989172,0.13559,0.102041,0.670762,0.545283,0.92457
needclothes,0.475639,0.414194,0.731362,0.463298,0.171272,0.279007,0.604304,0.73265,0.683329,0.551303,...,0.618119,0.289207,0.952464,0.309071,0.616896,0.601459,0.935432,0.71994,0.030898,0.95484
needshoes,0.250758,0.422931,0.826947,0.214561,0.518476,0.646686,0.323633,0.828863,0.152915,0.692908,...,0.236232,0.475503,0.801111,0.088744,0.706582,0.432403,0.738027,0.970472,0.467593,0.453375
needwood,0.425897,0.832711,0.317514,0.939592,0.70331,0.318885,0.141349,0.312952,0.692382,0.157826,...,0.408117,0.647143,0.671879,0.049186,0.53237,0.969845,0.106845,0.625057,0.397715,0.14221


Need: eatpro, Num species: 1700, Species effective for this need: ['specie_0', 'specie_1', 'specie_2', 'specie_3', 'specie_4', 'specie_5', 'specie_6', 'specie_7', 'specie_8', 'specie_9', 'specie_10', 'specie_11', 'specie_12', 'specie_13', 'specie_14', 'specie_15', 'specie_16', 'specie_17', 'specie_18', 'specie_19', 'specie_20', 'specie_21', 'specie_22', 'specie_23', 'specie_24', 'specie_25', 'specie_26', 'specie_27', 'specie_28', 'specie_29', 'specie_30', 'specie_31', 'specie_32', 'specie_33', 'specie_34', 'specie_35', 'specie_36', 'specie_37', 'specie_38', 'specie_39', 'specie_40', 'specie_41', 'specie_42', 'specie_43', 'specie_44', 'specie_45', 'specie_46', 'specie_47', 'specie_48', 'specie_49', 'specie_50', 'specie_51', 'specie_52', 'specie_53', 'specie_54', 'specie_55', 'specie_56', 'specie_57', 'specie_58', 'specie_59', 'specie_60', 'specie_61', 'specie_62', 'specie_63', 'specie_64', 'specie_65', 'specie_66', 'specie_67', 'specie_68', 'specie_69', 'specie_70', 'specie_71', 'specie

In [37]:
class Grid:
    """This class represents the grid where patches and foragers interact"""

    def __init__(self, foragers, memory, patches, beta, kmax):
        """This is what builds the grid. Called every time a new grid is constructed."""

        #print ("grid created")
        # initialize the variables
        self.numForagers = foragers
        self.memory = memory      
        self.numPatches = patches
        self.beta = beta
        self.kmax = kmax
        self.meanK = 0
        self.bagPatches=[]
        self.bagForagers = []
        

        # create patches
        self.createPatches()
        
        # create foragers
        self.createForagers()
        #----------------------------------------------------------------------------------------------------------

    def nextState(self,rad):
        """this function is in charge of changing the foragers and tress to a new state every new time-step"""
        # sumK = 0
        z = 0


        # loop for the next state of the patches
        while z < self.numPatches:
            self.bagPatches[z].nextState()

                
            # Add number of foods to meanK counter
            z = z + 1
        z = 0
        


        # loop for the next state of the foragers
        while z < self.numForagers:
            #print("forager", z)
            self.bagForagers[z].nextState(rad)
            z = z + 1
        z = 0
        string = ""
        
        while z < len(self.bagForagers):
            string = string + str(self.bagForagers[z].name)+","+ str(self.bagForagers[z].x)+","+str(self.bagForagers[z].y)+","+str(self.bagForagers[z].patch.k)+","
            
            z = z + 1
        return string+"\n"
    
    def output(self):
        string_dis = ""
        z = 0 
        while z < len(self.bagForagers):
            string_dis = string_dis + str(self.beta) +","+ str(self.bagForagers[z].name)+","+ str(self.bagForagers[z].dismoved)+","+ str(self.bagForagers[z].homemoved) +","+str(self.bagForagers[z].totalfoodinrad) +","+str(self.bagForagers[z].logmoves) +","+str(self.bagForagers[z].resmoves)+"\n"
#             #print(string_dis)
            z = z + 1
        return string_dis+"\n"
    
    def strats(self, csvwriter, step):        
        for forager in list(self.bagForagers):
            print(forager.name)
            print(type(forager.name))
            for category in self.bagForagers[int(forager.name)].agent_strategies:
                for need in self.bagForagers[int(forager.name)].agent_strategies[category]:
                    csvwriter.writerow([step, int(forager.name), self.beta, category, need, self.bagForagers[int(forager.name)].agent_strategies[category][need]])
        return csvwriter
        

        #----------------------------------------------------------------------------------------------------------  
            
    def createPatches(self):
        '''This method is in charge of creating the patches according to a beta distribution'''
        z = 1
        c = 0

        patch = 0 
        # patch counter

        # for the variable c from k=1 to k= kmax, we need to iterate
        while z <= self.kmax:
            c = c + pow(z, -self.beta)
            z = z +1
            
        # Loop for patch creating - one loop per patch
        # repeated until all patches are created
        while len(self.bagPatches) < self.numPatches:
            patch_num_species = num_species * species_per_patch
            patch_species = random.sample(species, patch_num_species)

            # try to create a patch for all distribution from k to kmax
            k = 1
       
            # there's a problem with k = 1
            while (k <= self.kmax) & (len(self.bagPatches) < self.numPatches):
                c = c + pow(k, -self.beta)
                pk = pow(k, -self.beta)/c
                z = 0
     
                # try a certain amount of times to create a patch from a specific k 
                while (z <self.numPatches )&( len(self.bagPatches) < self.numPatches) :
                    z = z + 1
                    azar = random.random()
                
                    # if the probability is enough, then the patch is created
                    if azar < pk:
                        # patches are random points on the grid, which is a unit square of 1x1
                        # there are 50,000 random patches, as set by "patches" in variables
                        self.bagPatches.append(Patch(random.random(), random.random(), k, patch, patch_species))
                        patch = patch + 1

                k = k + 1
    #----------------------------------------------------------------------------------------------------------
        
    def createForagers(self):
        """This method creates a certain amount of foragers, randomly distributed, a set of x,y coordinates, and a memory k"""
        z = 0
        dismoved = []
        homemoved = []
        foodinrad = 0
        totalfoodinrad = []
        agent_strategies = {}
        tried_species = []
        # el loop de ceaccion
        # forager creation loop
        while z < self.numForagers :
            self.bagForagers.append(Forager( z , self.givePatchesMemory(), agent_strategies, tried_species, self, dismoved, homemoved,foodinrad, totalfoodinrad))
            z = z + 1
        #----------------------------------------------------------------------------------------------------------

    def givePatchesMemory(self):
        "This method generates and returns a set of memory patches for a forager"

        # fill container of size equal to self.memory
        memories = []

        # obtain a copy of the set of patches to which we can substract the chosen patches
        bagPatches2 = self.bagPatches[:]
        while len(memories)  < self.memory: # self.memory controled by 'memory' number in variable setting at top
            # pick a random patch
            azar = random.randint(0 , (len(bagPatches2)-1))
            # turn it into memory 
            memories.append((bagPatches2[azar].x,bagPatches2[azar].y,bagPatches2[azar].k))
            # remove it from the set of patches 
            del bagPatches2[azar]
            

        return memories
    
    def givePatchEnPos(self, x, y):

        # This method helps locate a patch object based on its coordinates: x, y and k
        
        z=0
        # Search comparing each patch in the set of patches with the given coordinates
        while z < len(self.bagPatches):
            if (self.bagPatches[z].x == x) & (self.bagPatches[z].y == y):
                return self.bagPatches[z]
            z = z + 1

            

In [38]:
class Forager:
    """This class represents a forager with all its attributes and functions"""

    def __init__(self, name, memories, agent_strategies, tried_species, grid, dismoved, homemoved, foodinrad, totalfoodinrad):
        '''Create a forager in a specific position, with a name, and a memory'''
        
        # initialize variables 
        self.name = name
        self.memories = memories

        self.tried_species = tried_species
              ## change tried species so it's initialized here as just (all species - species in agent_strategies)
        self.grid = grid
        azar = random.randint(0, (len(self.memories) - 1 ))
        self.patch = self.grid.givePatchEnPos(self.memories[azar][0],self.memories[azar][1])
        self.homePatch = self.patch
        self.x = self.patch.x
        self.y = self.patch.y
        self.removeMemory(self.patch)
        self.patch.putGuest()
        self.kfRadius = 0 
#         self.kfRadius = self.check_radius_forfood()
        self.dismoved = []
        self.homemoved = []
        self.totalfoodinrad = []
        self.foodinrad = 0
        self.resmoves = 0
        self.logmoves = 0 
        self.previousheading = None
#         ##print(vars(self))

        # initialied here so that it doesn't get inherited by other foragers
        self.agent_strategies = {
               "EAT": {
                   "eatpro": [],
                   "eatfat": [],
                   "eatcarb": [],
               },
               "ILLNESS": {
                   "treatpain": [],
                   "treatwound": [],
                   "treatfever": [],
               },
               "SHELTER": {
                   "needclothes": [],
                   "needshoes": [],
                   "needwood": [],
               },
               "TOOLS": {
                   "needbasket": [],
                   "needaxe": [],
                   "needarrow": [],
               }
           }
        #----------------------------------------------------------------------------------------------------------

    def nextState(self, rad):
        """Make sure they all start from homePatch"""
        
#         print(self.x, self.y, self.previousheading)

        r = rad
        homePatch = self.homePatch
        patch = self.patch
        self.x = patch.x
        self.y = patch.y
        #logging.info(f'Forager {self.name}, {self.x, self.y}')

        ## each day, try 1 new specie for a need
        
        # if ANY need has no strategy yet
        logging.info(self.name, self.agent_strategies)
        if any_list_empty(self.agent_strategies):
            species_to_try = species
            # randomly sample one from the species in that patch
            selected_specie = random.sample(species_to_try, 1)[0] # the [0] makes it a string, instead of a list

            # loop through needs in need_species_dict and look for selected_specie
            for need in need_species_dict:
                if selected_specie in need_species_dict[need]["species"]:
                        specie_category = key_to_category[need]
                        self.agent_strategies[specie_category][need].append(selected_specie)
                        logging.info(f'Specie {selected_specie} effective for {specie_category}, {need}')
                        logging.info(self.name, self.agent_strategies)



        if self.patch.k <= 0 :
            self.removeMemory(self.patch)
#             start = timeit.default_timer()
            self.kfRadius = self.check_radius_forfood(r) # only returns one set -- best/closest option, maximises benefit/cost ratio 
#             print(timeit.default_timer() - start)
        # See food availavility within your fRadius
#             print("food in radius?", self.kfRadius)
           
        # If there is any food within your fRadius, you forage
            if self.kfRadius != None:

                # change coordinates
                self.x = self.kfRadius[0]
                self.y = self.kfRadius[1]
                self.foodinrad = self.foodinrad + 1
    #             ##print(self.foodinrad)
    
                # move from the old patch and get to the new one
                patch = self.grid.givePatchEnPos(self.x , self.y)
                self.patch.removeGuest()
                patch.putGuest()
                self.dismoved.append(round(self.kfRadius[2],5))
                # Here DO NOT remove patch (as not depleted) from memory
                self.patch = patch
                self.logmoves += 1 

                
                                

            # If no food within your fRadius, you change homePatch
            else : 
                logging.info(f'No food in radius')
                # Examine distances and Ks 
                self.totalfoodinrad.append(self.foodinrad)
                self.foodinrad = 0
                self.x, self.y, d = self.check_forhome_avoidoverlap(r)
                self.kfRadius = 0
                self.homemoved.append(round(d,5))
                logging.info(f'Moved to {round(d,5)}')
                # move from the old patch and get to the new one
                patch = self.grid.givePatchEnPos(self.x , self.y)
                self.patch.removeGuest()
                self.homePatch.removeGuest()
                patch.putGuest()
                # Remove depleted patch from memory
                self.removeMemory(patch)
                self.patch = patch
                self.homePatch = patch
                homePatch = patch
                self.resmoves += 1 
            
        #----------------------------------------------------------------------------------------------------------
        
    def removeMemory(self, patch):
        # This method substracts the patch from memory 
        z = 0
        # search for the patch coordinates to remove it from memory 
        
        while z < len(self.memories):
            if (patch.x == self.memories[z][0])&(patch.y == self.memories[z][1]):
                del self.memories[z]
                break
            z = z + 1
        #----------------------------------------------------------------------------------------------------------
    
    def check_radius_forfood(self,r):
        
        cb_home = []
        dis=[]
#         num = []
        z= 0 
        
        # recall that self.memories[memory][0] is x coordinate of that patch in memory, and [1] is y coordinate, and [2] is k (# food)
        # loop over all memories

        while z < len(self.memories) :
            # get the distance from home patch to each patch in memories (using pythagorean theorem)
            distanceHZ =  math.sqrt(pow(self.homePatch.x - self.memories[z][0],2) + pow(self.homePatch.y - self.memories[z][1],2))
            # append the 'distance to that patch'
            dis.append(distanceHZ)
            # append the value formula -- (distance divided by 'k of that patch')
            cb_home.append((distanceHZ )/self.memories[z][2] )
                # we're going to look for the minimum using this
                # so farther distance = bigger cb_home entry = bigger cost
                # but smaller distance = lower cb_home entry = lower cost
                # and greater K = lower cb_home entry = lower cost (because higher reward)

            z = z + 1 

        # now dis is a list of the distance to all the patches in self.memories
        # and cb_home is a list of all the values of all the patches in self.memories
        #########
        best_cb = np.argmin(cb_home)  ##get index of minimum cost/benefit from above
        ##use index to find the patch
        to_return = [self.memories[best_cb][0], self.memories[best_cb][1], dis[best_cb]]
        # if the distance to the best patch is within radius, return [x, y, k]
        if dis[best_cb] <= r : 
            return to_return 
        # else, return None so you look outside your radius
        else :
            return None 
        
        
    
       
    def check_forhome_avoidoverlap(self,r):
        cb = []
        num=[]
        diss=[]
#         ks=[]
#         z= 0 

        # recall that self.memories[memory][0] = x coordinate of that patch in memory, and [1] is y coordinate, and [2] is k (# food)
        # and er is set at top in variable settings, and is 'effective radius' for home 

        # loop over all memories
        for memory in range(len(self.memories)) :
            # get the distance from home patch to each patch in memories (using pythagorean theorem)
            dis = math.sqrt(pow(self.homePatch.x - self.memories[memory][0],2) + pow(self.homePatch.y - self.memories[memory][1],2))
            # if the distance to that patch is greater than the radius (times effective radius aka times 2)
            if dis > (er * r) :
                # append the 'distance to that patch'
                diss.append(dis)
                # append the value formula-- distance divided by 'k of that patch'
                cb.append(dis / self.memories[memory][2] )
                # append the whole 'patch' from memory
                num.append(memory)
#                 ks.append()
                
        #########
        best_cb = np.argmin(cb)
        best_home = num[best_cb]
    
        return self.memories[best_home][0], self.memories[best_home][1], diss[best_cb]
        
    
    
   ############################### End of class forager ###############################
                                    
class Patch: # This class represents a patch with all functions and attributes
    # create a patch with a size k, a position, and a name
    def __init__(self, x, y, k, name, species):
        # This method is called for creating a new patch

        # initialize the variables
        self.x = x
        self.y = y
        self.k = k
        self.name = name
        self.ocupantes = 0
        self.species = species
        #----------------------------------------------------------------------------------------------------------
        
    def nextState(self):
        # This function calculates the next state of the patch based on how its k has changed
        if self.ocupantes > 0:
            self.removeFood()
        #----------------------------------------------------------------------------------------------------------
        
    def putGuest(self):
        # This method is called when a guest jumps on the patch
        self.ocupantes = self.ocupantes + 1
        #----------------------------------------------------------------------------------------------------------
        
    def removeGuest(self):
        # This method is called when a guest leaves
        self.ocupantes = self.ocupantes - 1
        #----------------------------------------------------------------------------------------------------------
        
    def removeFood(self):
        """This method is used to substract the foods eaten from a patch"""
        if self.k > 0:
            self.k = self.k - self.ocupantes
        ##print("decrease food", self.name, self.k, self.ocupantes, self.x, self.y)
        #----------------------------------------------------------------------------------------------------------
        

Current time: Thu May 22 21:37:53 2025


In [ ]:
def main():
    start_time = time.time()
    """esta es la funcion principal donde comienza la ejecucion del
    programa
    """
    # This is the main function that executes the program
    folder = mainFolder + f"_{start_time}"
    
    # create directory for saving all the files
    os.makedirs(folder, exist_ok=True)
    
    top_folder = 'beta'
    # set initial beta and iterate until the max value of beta
    for beta in betas :
        

        # create a subdirectory for each value of beta       
        os.makedirs(mainFolder+f"/{top_folder}"+str(beta), exist_ok=True)

        #por cada beta provado obten un kmax
        # for every beta that has been tested get a kmax
        kmax = giveKmax(patches, beta)
        

        # for each beta try a range of number of foragers (from a starting value to a ceiling)
        for forager in foragers:
            # create a subdirectory for each one of the different number of foragers
            os.makedirs(mainFolder+f"/{top_folder}"+str(beta)+"/foragers"+str(forager),exist_ok=True)
            
            for rad in fRadius :
                os.makedirs(mainFolder+f"/{top_folder}"+str(beta)+"/foragers"+str(forager)+"/radius"+str(rad), exist_ok=True)

            # for each one of the runs pick a specific beta and number of foragers
            # run the program for a certain amount of iterations with the same initial conditions
                z = 0
                while z < runs:
                    print (f"{top_folder} = "+str(beta)+" foragers = "+str(forager)+" radius=" +str(rad)+"corrida = "+str(z))
                    f=open(mainFolder+f"/{top_folder}"+str(beta)+"/foragers"+str(forager)+"/radius"+str(rad)+"/"+str(z)+f"_{start_time}.txt", 'w')
                    fdis = open(mainFolder+f"/{top_folder}"+str(beta)+"/foragers"+str(forager)+"/radius"+str(rad)+"/"+str(z)+f"_{start_time}_dis.txt", 'w')
                    fstrat=mainFolder+f"/{top_folder}"+str(beta)+"/foragers"+str(forager)+"/radius"+str(rad)+"/"+str(z)+f"_{start_time}_strat.csv"
                    strat = open(fstrat, 'w', newline='')
                    strat = csv.writer(strat, delimiter='\t')
                    ################### here's where the whole program is executed #######
                    print ("---- grid ----")
                    grid= Grid(forager, memory, patches, beta, kmax)
                    print ("----------------------------------------")
                    step = 0
                    # Main cycle of each iteration
                    while step < steps:
                        print("timestep", step)
                        # call the function to create a string of the next state of the grid, the foragers, and patches
                        result = grid.nextState(rad)
                        f.write( result )

                        # strats = grid.strats()
                        # fstrat.write( strats )

                        strat = grid.strats(strat, step)

                        
                        step = step + 1
                    dis_output=grid.output()
                    fdis.write(dis_output)
                    f.close()
                    fdis.close()
                    ###########################################################################
                    z = z+1


            
#         fig = plt.figure()

#         ax1 = fig.add_subplot(2,2,1)
# #             fig1, ax1 = plt.subplots()

#         ax1.hist(all_movdis)
#         ax1.xscale('log')
#         all_movdis=[]
    #----------------------------------------------------------------------------------------------------------

def giveKmax(numero_de_patches, beta):
    ## kmax represents the patch with the highest amount of foods possible 
    ## this function generations an appropriate distribution around a good kmax
    # initialize the variables 
    z = 1
    norm = 0
    pkmax = 0
    # search for Kmax until you find it 
    while True:      
        norm = norm + pow(z , -beta)
        pkmax = numero_de_patches * pow(z , -beta)/norm        
        if pkmax  <= 1.0 :
            print ("kmax = "+str(z))
            return z 
        z =  z + 1
        #----------------------------------------------------------------------------------------------------------      





           
# if this module has a main, call it    
if __name__ == '__main__': main()
    



kmax = 68
beta = 2.5 foragers = 100 radius=0.1corrida = 0
---- grid ----
----------------------------------------
timestep 0
0
<class 'int'>
1
<class 'int'>
2
<class 'int'>
3
<class 'int'>
4
<class 'int'>
5
<class 'int'>
6
<class 'int'>
7
<class 'int'>
8
<class 'int'>
9
<class 'int'>
10
<class 'int'>
11
<class 'int'>
12
<class 'int'>
13
<class 'int'>
14
<class 'int'>
15
<class 'int'>
16
<class 'int'>
17
<class 'int'>
18
<class 'int'>
19
<class 'int'>
20
<class 'int'>
21
<class 'int'>
22
<class 'int'>
23
<class 'int'>
24
<class 'int'>
25
<class 'int'>
26
<class 'int'>
27
<class 'int'>
28
<class 'int'>
29
<class 'int'>
30
<class 'int'>
31
<class 'int'>
32
<class 'int'>
33
<class 'int'>
34
<class 'int'>
35
<class 'int'>
36
<class 'int'>
37
<class 'int'>
38
<class 'int'>
39
<class 'int'>
40
<class 'int'>
41
<class 'int'>
42
<class 'int'>
43
<class 'int'>
44
<class 'int'>
45
<class 'int'>
46
<class 'int'>
47
<class 'int'>
48
<class 'int'>
49
<class 'int'>
50
<class 'int'>
51
<class 'int'>
5

KeyboardInterrupt: 

took 29min to do 1 run for all betas on 100 foragers normal version
took 94 minutes to run 1 beta for 1000 timesteps on 100 foragers